# 기능: 질환 예측하기

In [1]:
import openai
import os

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## ChatCompletion API
Role
- user
- assistant (ai)
- system (ai에게 역할 부여-성격, 말투)

In [3]:
response = openai.ChatCompletion.create(
    model="gpt-4-turbo",
    messages=[
        {
            "role": "user",
            "content": "안녕!"
        }
    ]
)

In [4]:
response["choices"][0]["message"]["content"]

'안녕하세요! 무엇을 도와드릴까요?'

In [5]:
###모델 파라미터

In [3]:
def request_chat_completion(
    prompt, 
    system_role="당신은 사용자의 상황에 따라 질환명을 예측해주는 유용한 질환예측 도우미입니다.당신은 유용한 도우미입니다.", 
    model="gpt-4-turbo",
    stream=False
):
    messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": prompt}
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        stream=stream
    )
    return response

In [4]:
def print_streaming_response(response):
    for chunk in response:
        delta = chunk.choices[0]["delta"]
        if "content" in delta:
            print(delta["content"], end="")
        else:
            break

## 프롬프트 엔지니어링

In [5]:
prompt_template = """
당신의 임무는 일관된 스타일로 답변하는 것입니다.
통증 부위: 허리, 목.
발생 시각: 2024.07.06.
오래 서 있을때 통증 발생.
통증 느낌: 저림
통증 강도: 2/10.
다음과 같은 통증 호소자에게 관련 질환 3개를 중요도 순으로 알려줘.
질환명만 적어.
""".strip()

In [6]:
print(prompt_template)

당신의 임무는 일관된 스타일로 답변하는 것입니다.
통증 부위: 허리, 목.
발생 시각: 2024.07.06.
오래 서 있을때 통증 발생.
통증 느낌: 저림
통증 강도: 2/10.
다음과 같은 통증 호소자에게 관련 질환 3개를 중요도 순으로 알려줘.
질환명만 적어.


In [10]:
system_role= "당신은 사용자의 상황에 따라 질환명을 예측해주는 유용한 질환예측 도우미입니다."
response = request_chat_completion(prompt_template, system_role=system_role, stream=True)

In [11]:
print_streaming_response(response)

1. 요통
2. 경추 디스크
3. 근막통증증후군

## 문제점
- 매번 결과가 다르게 나옴. 반복적으로 보이는 질환도 있지만 아닐 경우가 더 많음
- [결과1]
1. 요추염좌
2. 경추염좌
3. 추간판 탈출증
4. 척추관 협착증
5. 근막통증증후군
- [결과2]
1. 경추 디스크
2. 요추 디스크
3. 근막통증증후군
4. 척추 관절염
5. 척추협착증
- [결과3]
1. 요통
2. 경추 디스크
3. 근막통증증후군